In [540]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [541]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [542]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [543]:
%%capture pwd
!pwd

In [544]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('how-has-taiwan-navigated-the-pandemic', 300, 500)

# Fig 1

In [545]:
df = pd.read_excel(
    "raw/Exhxibits_Fig_1-5+Table_1_CHENG_How_has_Taiwan_navigated_the_Covid-19_pandemic_OCT_25.2021.xlsx",
    sheet_name="Fig 1 Cases Taiwan+OECD ",
    skiprows=2,
).dropna(axis=1, how="all")
df.columns = ["country", "cases"]
df = df.dropna(axis=0, subset=["cases"]).sort_values(by="cases", ascending=False)
df["cases"] = df["cases"].astype(int)

In [546]:
f = "fig1_cases"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,country,cases
19,Israel,14601
18,US,13825
17,UK,13002
16,Netherlands,11946
15,Sweden,11336


In [547]:
base = alt.Chart(f1).encode(
    y=alt.Y(
        "country:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    )
)
line = base.mark_bar(color=colors["eco-turquiose"], opacity=0.8).encode(
    x=alt.X(
        "cases:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="middle",
            title="COVID-19 cases per 100,000 population, on 22 October 2021",
            titleY=-15,
            titleX=400,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
labels = (
    base.mark_text(color=colors["eco-turquiose"], align="left", dx=5)
    .encode(
        x=alt.X(
            "cases:Q",
            sort=[],
        ),
        text=alt.Text("cases:Q", format=",.0f"),
    )
    .transform_filter("datum.country!='Taiwan'")
)
labels2 = (
    base.mark_text(color=colors["eco-dot"], align="left", dx=5)
    .encode(
        x=alt.X(
            "cases:Q",
            sort=[],
        ),
        text=alt.Text("cases:Q", format=",.0f"),
    )
    .transform_filter("datum.country=='Taiwan'")
)
layer1 = (
    ((line + labels + labels2).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "w").write(readme)
layer1

alt.LayerChart(...)

# Fig 2

In [548]:
df = pd.read_excel(
    "raw/Exhxibits_Fig_1-5+Table_1_CHENG_How_has_Taiwan_navigated_the_Covid-19_pandemic_OCT_25.2021.xlsx",
    sheet_name="Fig 2 Deaths Taiwan+OECD",
    skiprows=1,
).dropna(axis=1, how="all")
df.columns = ["country", "cases"]
df = df.dropna(axis=0, subset=["cases"]).sort_values(by="cases", ascending=False)

In [549]:
f = "fig2_deaths"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,country,cases
18,US,224.0
19,Italy,219.0
17,UK,208.0
16,Spain,185.0
15,France,172.0


In [550]:
base = alt.Chart(f2).encode(
    y=alt.Y(
        "country:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    )
)
line = base.mark_bar(color=colors["eco-turquiose"], opacity=0.8).encode(
    x=alt.X(
        "cases:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="middle",
            title="COVID-19 deaths per 100,000 population, on 22 October 2021",
            titleY=-15,
            titleX=400,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
labels = (
    base.mark_text(color=colors["eco-turquiose"], align="left", dx=5)
    .encode(
        x=alt.X(
            "cases:Q",
            sort=[],
        ),
        text=alt.Text("cases:Q", format=",.0f"),
    )
    .transform_filter("datum.country!='Taiwan'")
)
labels2 = (
    base.mark_text(color=colors["eco-dot"], align="left", dx=5)
    .encode(
        x=alt.X(
            "cases:Q",
            sort=[],
        ),
        text=alt.Text("cases:Q", format=".0f"),
    )
    .transform_filter("datum.country=='Taiwan'")
)
layer1 = (
    ((line + labels + labels2).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 3

In [551]:
df = pd.read_excel(
    "raw/Exhxibits_Fig_1-5+Table_1_CHENG_How_has_Taiwan_navigated_the_Covid-19_pandemic_OCT_25.2021.xlsx",
    sheet_name="Fig 3 Tw Ec Growth 2013-2022",
    skiprows=2,
).dropna(axis=1, how="all")
df.columns = ["year", "growth"]
df = df.dropna(axis=0, subset=["growth"])
df["growth"] /= 100.0

In [552]:
f = "fig3_growth"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,growth
0,2013,0.0223
1,2014,0.0392
2,2015,0.0081
3,2016,0.0150
4,2017,0.0286


In [553]:
base = alt.Chart(f3).encode(
    x=alt.X(
        "year:N",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=5,
            labelOffset=-10,
        ),
    ),
    y=alt.Y(
        "growth:Q",
        sort=[],
        axis=alt.Axis(
            grid=True,
            gridColor=colors["eco-gray"],
            gridOpacity=0.1,
            title="GDP growth",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=25,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=7,
            format=".0%",
        ),
    ),
)
spacer = (
    alt.Chart(pd.DataFrame([{"year": " ", "growth": 0}]))
    .mark_bar()
    .encode(y="growth:Q", x="year:N")
)
bars = base.mark_bar(
    color=colors["eco-turquiose"], opacity=0.8, width=20
).transform_filter("datum.year<2020")
text = base.encode(text=alt.Text("growth:Q", format=".1%"))
text1 = text.mark_text(color=colors["eco-turquiose"], dy=-10).transform_filter(
    "datum.year<2020"
)
bars1 = base.mark_bar(
    color=colors["eco-light-blue"], opacity=0.8, width=20
).transform_filter(alt.FieldOneOfPredicate(field="year", oneOf=[2020]))
text2 = text.mark_text(color=colors["eco-light-blue"], dy=-10).transform_filter(
    alt.FieldOneOfPredicate(field="year", oneOf=[2020])
)
bars2 = base.mark_bar(color=colors["eco-blue"], opacity=0.8, width=20).transform_filter(
    alt.FieldOneOfPredicate(field="year", oneOf=[2021, 2022])
)
text3 = text.mark_text(color=colors["eco-blue"], dy=-10).transform_filter(
    alt.FieldOneOfPredicate(field="year", oneOf=[2021, 2022])
)
text12 = base.mark_text(
    color=colors["eco-light-blue"],
    dy=-25,
    dx=5,
    text="start of the pandemic ➡",
    align="right",
).transform_filter(alt.FieldOneOfPredicate(field="year", oneOf=[2020]))
text13 = base.mark_text(
    color=colors["eco-blue"], dy=15, dx=-15, text="prediction ➡", align="right"
).transform_filter(alt.FieldOneOfPredicate(field="year", oneOf=[2021]))
layer1 = (
    (
        (
            spacer + bars + bars1 + bars2 + text1 + text2 + text3 + text12 + text13
        ).properties(height=250, width=alt.Step(30))
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 4

In [554]:
df = pd.read_excel(
    "raw/Exhxibits_Fig_1-5+Table_1_CHENG_How_has_Taiwan_navigated_the_Covid-19_pandemic_OCT_25.2021.xlsx",
    sheet_name="Fig 4 EC IMPACT_GDP OECD+Taiwan",
    skiprows=1,
).dropna(axis=1, how="all")
df.columns = ["country", "growth"]
df = df.dropna(axis=0, subset=["growth"])
df["growth"] /= 100.0

In [555]:
f = "fig4_growth_world"
f4 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,country,growth
0,World,-0.034
1,G20,-0.032
2,Euro area,-0.068
3,Germany,-0.053
4,France,-0.082


In [556]:
base = alt.Chart(f4).encode(
    x=alt.X(
        "country:N",
        sort=[],
        axis=None,
    ),
    y=alt.Y(
        "growth:Q",
        sort=[],
        axis=alt.Axis(
            grid=True,
            gridColor=colors["eco-gray"],
            gridOpacity=0.1,
            title="GDP growth in 2020",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=25,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=7,
            format=".0%",
        ),
    ),
)
spacer = (
    alt.Chart(pd.DataFrame([{"country": " ", "growth": 0}]))
    .mark_bar()
    .encode(y="growth:Q", x=alt.X("country:N", sort=[]))
)
bars = base.mark_bar(color=colors["eco-dot"], opacity=0.8, width=20).transform_filter(
    "datum.country!='Taiwan'"
)
text = base.encode(text=alt.Text("growth:Q", format=".1%"))
text1 = text.mark_text(color=colors["eco-dot"], dy=10).transform_filter(
    "datum.country!='Taiwan'"
)
bars1 = base.mark_bar(
    color=colors["eco-light-blue"], opacity=0.8, width=20
).transform_filter("datum.country=='Taiwan'")
text2 = text.mark_text(color=colors["eco-light-blue"], dy=-10).transform_filter(
    "datum.country=='Taiwan'"
)
text3 = (
    alt.Chart(f4)
    .encode(text=alt.Text("country:N"), x=alt.X("country:N", sort=[]))
    .mark_text(color=colors["eco-dot"], dx=70, angle=270, align="left")
    .transform_filter("datum.country!='Taiwan'")
)
text4 = (
    alt.Chart(f4)
    .encode(text=alt.Text("country:N"), x=alt.X("country:N", sort=[]))
    .mark_text(color=colors["eco-light-blue"], dx=55, angle=270, align="right")
    .transform_filter("datum.country=='Taiwan'")
)
layer1 = (
    (
        (spacer + bars + bars1 + text1 + text2 + text3 + text4).properties(
            height=250, width=alt.Step(30)
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
# layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
# layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 5

In [557]:
df = (
    pd.read_excel(
        "raw/Exhxibits_Fig_1-5+Table_1_CHENG_How_has_Taiwan_navigated_the_Covid-19_pandemic_OCT_25.2021.xlsx",
        sheet_name="Fig 5 Growth NZ_Aus_Japan_K_Tw ",
        skiprows=2,
    )
    .dropna(subset=["Unnamed: 1"])
    .dropna(axis=1, how="all")
)
df = df.set_index("Unnamed: 1").stack().reset_index()
df.columns = ["country", "year", "value"]
df["value"] /= 100.0

In [558]:
f = "fig5_projections"
f5 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f5.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f5 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,country,year,value
0,Taiwan,2020,0.0311
1,Taiwan,2021,0.0588
2,Taiwan,2022,0.0369
3,Australia,2020,-0.0240
4,Australia,2021,0.0350


In [559]:
base = alt.Chart(f5).encode(
    x=alt.X(
        "country:N",
        sort=[],
        axis=None,
    ),
    y=alt.Y(
        "value:Q",
        sort=[],
        axis=alt.Axis(
            grid=True,
            gridColor=colors["eco-gray"],
            gridOpacity=0.1,
            title="GDP growth in 2020 and projections for 2021 and 2022",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=25,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=7,
            format=".0%",
        ),
    ),
)
spacer = (
    alt.Chart(pd.DataFrame([{"country": " ", "growth": 0}]))
    .mark_bar()
    .encode(y="growth:Q", x=alt.X("country:N", sort=[]))
)
bars1 = (
    base.mark_bar(color=colors["eco-dot"], opacity=0.8, width=15)
    .transform_filter("datum.country!='Taiwan'")
    .transform_filter("datum.year==2020")
)
bars2 = (
    base.mark_bar(color=colors["eco-dot"], opacity=0.8, width=15, xOffset=18)
    .transform_filter("datum.country!='Taiwan'")
    .transform_filter("datum.year==2021")
)
bars3 = (
    base.mark_bar(color=colors["eco-dot"], opacity=0.8, width=15, xOffset=36)
    .transform_filter("datum.country!='Taiwan'")
    .transform_filter("datum.year==2022")
)
text = base.encode(text=alt.Text("value:Q", format=".1%"))
text1 = (
    text.mark_text(color=colors["eco-dot"], dy=10)
    .transform_filter("datum.country!='Taiwan'")
    .transform_filter("datum.year==2020")
)
text1c = (
    text.mark_text(color=colors["eco-dot"], dy=-10, dx=14)
    .transform_filter("datum.country!='Taiwan'")
    .transform_filter("datum.year==2021")
)
text1d = (
    text.mark_text(color=colors["eco-dot"], dy=-10, dx=40)
    .transform_filter("datum.country!='Taiwan'")
    .transform_filter("datum.year==2022")
)
bars1b = (
    base.mark_bar(color=colors["eco-light-blue"], opacity=0.8, width=15)
    .transform_filter("datum.country=='Taiwan'")
    .transform_filter("datum.year==2020")
)
bars2b = (
    base.mark_bar(color=colors["eco-light-blue"], opacity=0.8, width=15, xOffset=18)
    .transform_filter("datum.country=='Taiwan'")
    .transform_filter("datum.year==2021")
)
bars3b = (
    base.mark_bar(color=colors["eco-light-blue"], opacity=0.8, width=15, xOffset=36)
    .transform_filter("datum.country=='Taiwan'")
    .transform_filter("datum.year==2022")
)
text2 = (
    text.mark_text(color=colors["eco-light-blue"], dy=-10, dx=-3)
    .transform_filter("datum.country=='Taiwan'")
    .transform_filter("datum.year==2020")
)
text2c = (
    text.mark_text(color=colors["eco-light-blue"], dy=-10, dx=18)
    .transform_filter("datum.country=='Taiwan'")
    .transform_filter("datum.year==2021")
)
text2d = (
    text.mark_text(color=colors["eco-light-blue"], dy=-10, dx=40)
    .transform_filter("datum.country=='Taiwan'")
    .transform_filter("datum.year==2022")
)
text3 = (
    alt.Chart(f5)
    .encode(text=alt.Text("country:N"), x=alt.X("country:N", sort=[]))
    .mark_text(color=colors["eco-dot"], dx=-5, angle=270, align="left")
    .transform_filter("datum.country!='Taiwan'")
    .transform_filter("datum.year==2020")
)
text4 = (
    alt.Chart(f5)
    .encode(text=alt.Text("country:N"), x=alt.X("country:N", sort=[]))
    .mark_text(color=colors["eco-light-blue"], dx=-15, angle=270, align="right")
    .transform_filter("datum.country=='Taiwan'")
    .transform_filter("datum.year==2020")
)
label1 = (
    alt.Chart(pd.DataFrame([{"x": "Taiwan", "y": 0, "t": "2020"}]))
    .encode(text=alt.Text("t:N"), x=alt.X("x:N", sort=[]), y="y:Q")
    .mark_text(color="white", dx=4, angle=270, dy=1, align="left")
)
label2 = (
    alt.Chart(pd.DataFrame([{"x": "Taiwan", "y": 0, "t": "2021"}]))
    .encode(text=alt.Text("t:N"), x=alt.X("x:N", sort=[]), y="y:Q")
    .mark_text(color="white", dx=4, angle=270, dy=19, align="left")
)
label3 = (
    alt.Chart(pd.DataFrame([{"x": "Taiwan", "y": 0, "t": "2022"}]))
    .encode(text=alt.Text("t:N"), x=alt.X("x:N", sort=[]), y="y:Q")
    .mark_text(color="white", dx=4, angle=270, dy=37, align="left")
)
layer1 = (
    (
        (
            bars1
            + bars2
            + bars3
            + bars1b
            + bars2b
            + bars3b
            + text1
            + text2
            + text3
            + text4
            + text1c
            + text1d
            + text2c
            + text2d
            + label1
            + label2
            + label3
        ).properties(height=250, width=alt.Step(70))
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
# layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
# layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 6

In [560]:
df = pd.read_csv("raw/Dataset1.csv", header=None)
df = df.groupby(0).max()
df = df.sort_values(by=0).rolling(3).mean()
df[1] = [max(0, i) for i in df[1].values]
df = df.reset_index()
df.columns = ["date", "value"]

In [561]:
f = "fig6_outbreak"
f6 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f6.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f6 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,date,value
0,2021/05/04,0.0
1,2021/05/07,0.0
2,2021/05/09,0.0
3,2021/05/11,0.0
4,2021/05/12,0.0


In [562]:
base = alt.Chart(f6).encode(
    x=alt.X(
        "date:T",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line1 = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-mid-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-mid-blue"], offset=1),
        ],
        x1=1,
        x2=1,
        y1=0.9,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "value:Q",
        axis=alt.Axis(
            grid=True,
            title="Nr. of cases",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            #             labelAlign="left",
            #             labelBaseline="middle",
            #             labelPadding=-5,
            #             labelOffset=-10,
            titleX=5,
            titleY=20,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
    ),
)
layer1 = (
    ((line1).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

alt.Chart(...)

# Fig 7

In [563]:
df = pd.read_csv("raw/Dataset2.csv", header=None)
df = df.groupby(0).max()
df = df.sort_values(by=0).rolling(3).mean()
df[1] = [max(0, i) for i in df[1].values]
df = df.reset_index()
df.columns = ["date", "value"]

In [564]:
f = "fig7_changes"
f7 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f7.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f7 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,date,value
0,2021/05/13,0.000000
1,2021/05/14,0.000000
2,2021/05/15,2.646678
3,2021/05/17,55.022660
4,2021/05/18,113.474399


In [565]:
base = alt.Chart(f7).encode(
    x=alt.X(
        "date:T",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line1 = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-mid-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-mid-blue"], offset=1),
        ],
        x1=1,
        x2=1,
        y1=0.9,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "value:Q",
        axis=alt.Axis(
            grid=True,
            title="Change in the nr. of cases",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            #             labelAlign="left",
            #             labelBaseline="middle",
            #             labelPadding=-5,
            #             labelOffset=-10,
            titleX=-20,
            titleY=-10,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
    ),
)
label_data = alt.Chart(
    pd.DataFrame([{"x": "May 19", "y": 580, "t": "➡ National Level-III Alert"},{"x": "May 11", "y": 45, "t": "➡ National Level-II Alert"},
                 {"x": "May 15", "y": 320, "t": "➡ Taipei Level-III Alert"},{"x": "July 13", "y": 160, "t": "➡ Moderation of Level-III Alert"},
                 {"x": "July 27", "y": 70, "t": "➡ National Level-II Alert"}])
).transform_calculate(x2="toDate('2021 '+datum.x)")
labels = label_data.mark_text(color=colors["eco-gray"], align="left", dx=0,fontSize=10,dy=5).encode(
    x="x2:T", y="y:Q", text="t:N"
)
labels2 = label_data.mark_text(color=colors["eco-gray"], align="left", dx=13,fontSize=10,dy=17).encode(
    x="x2:T", y="y:Q", text="x:N"
)
rulers = label_data.mark_rule(color=colors["eco-gray"],strokeDash=[5,1],opacity=0.7).encode(x="x2:T",y="y:Q")
layer1 = (
    ((line1 + rulers + labels+labels2).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)